# Finetune RuGPTs in huggingface
How to finetune RuGPTs models with huggingface. Example for RuGPT3Small. Nfor other models it will take more GPU memory.

This notebook is valid for all RuGPTs models except RuGPT3XL.
## Install env

In [ ]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  binfmt-support libclang-common-9-dev libclang-cpp9 libclang1-9 libffi-dev
  libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9-runtime python-chardet
  python-pkg-resources python-pygments python-yaml python3-pkg-resources
  python3-pygments python3-yaml
Suggested packages:
  clang-9-doc libomp-9-doc llvm-9-doc python-setuptools ttf-bitstream-vera
  python3-setuptools
The following NEW packages will be installed:
  binfmt-support clang-9 libclang-common-9-dev libclang-cpp9 libclang1-9
  libffi-dev libllvm9 libomp-9-dev libomp5-9 libpfm4 llvm-9 llvm-9-dev
  llvm-9-runtime llvm-9-tools python-chardet python-pkg-resources
  python-pygments python-yaml python3-pkg-resources python3-pygments
  python3-yaml
0 upgra

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


If code below doesn't work, check your cuda version and installation here https://pytorch.org/get-started/previous-versions/

In [ ]:
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |████████████████████████████████| 735.4 MB 15 kB/s 
     |████████████████████████████████| 12.8 MB 39.5 MB/s 
     |████████████████████████████████| 7.6 MB 32.7 MB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1+cu113
    Uninstalling torch-1.12.1+cu113:
      Successfully uninstalled torch-1.12.1+cu113
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.13.1+cu113
    Uninstalling torchvision-0.13.1+cu113:
      Successfully uninstalled torchvision-0.13.1+cu113
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 0.12.1+cu113
    Uninstalling torchaudio-0.12.1+cu113:
      Successfully uninstalled torchaudio-0.12.1+cu113
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed.

In [ ]:
!pip install transformers==3.5.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 35.6 MB/s 
     |████████████████████████████████| 880 kB 61.0 MB/s 
     |████████████████████████████████| 2.9 MB 58.6 MB/s 
     |████████████████████████████████| 1.1 MB 60.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=24e6e038dcc74c15e5515770a68144166eba69f86b9ccb63f3f3c6a38d6743e9
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 686, done.
remote: Counting objects: 100% (244/244), done.
remote: Compressing objects: 100% (79/79), done.
remote: Total 686 (delta 189), reused 165 (delta 165), pack-reused 442
Receiving objects: 100% (686/686), 377.90 KiB | 1.64 MiB/s, done.
Resolving deltas: 100% (427/427), done.


In [ ]:
!mkdir models/

## Download files

In [ ]:
!pip install gdown

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!gdown --folder https://drive.google.com/drive/folders/1MQ3XmOl-lYyJwfmyvcFNEMCKMp_kRJko

Retrieving folder list
Processing file 124V_Fz_Rjp4qHGdE9h8Pnyu3-0CjbTYV race_ru_test.csv
Processing file 1uHl3F4GsonPcQ2mSvOGzlFUSpmhgdu-g race_ru_train.csv
Processing file 1CQpBK4SVW6D9xAsZo0KPj16yfXuLP8EB race_ru_valid.csv
Retrieving folder list completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=124V_Fz_Rjp4qHGdE9h8Pnyu3-0CjbTYV
To: /content/race_ru_test.csv
100% 24.6M/24.6M [00:00<00:00, 48.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1uHl3F4GsonPcQ2mSvOGzlFUSpmhgdu-g
To: /content/race_ru_train.csv
100% 441M/441M [00:07<00:00, 59.9MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CQpBK4SVW6D9xAsZo0KPj16yfXuLP8EB
To: /content/race_ru_valid.csv
100% 24.2M/24.2M [00:00<00:00, 70.3MB/s]
Download completed


In [ ]:
import os
os.listdir()

['.config',
 'race_ru_train.csv',
 'ru-gpts',
 'race_ru_test.csv',
 'models',
 'race_ru_valid.csv',
 'sample_data']

## Prepare data

In [ ]:
import pandas as pd
from ast import literal_eval

In [ ]:
def prepare_data_for_rugpt3(
    path: str,
    subset_len: int = None,
):
  txt = ""
  df = pd.read_csv(path, index_col="Unnamed: 0")
  fn = path.split(".")[0]
  df = df[["example_id","article_ru","question_ru","options_ru","answer"]]
  if subset_len is not None:
    df = df.iloc[:subset_len]
  for idx, article, question, options, answer in df.values:
    txt += f"ТЕКСТ: {article.replace(chr(10),' ')} ВОПРОС: {question.replace(chr(10),' ')} ВАРИАНТЫ: {options.replace(chr(10),' ')} ОТВЕТ: {answer}"
    txt += "\n"
  with open(f"{fn}.txt", 'w', encoding='utf-8') as outp:
    outp.write(txt)

In [ ]:
prepare_data_for_rugpt3("race_ru_train.csv", 8000)

In [ ]:
prepare_data_for_rugpt3("race_ru_valid.csv", 2000)

In [ ]:
from google.colab import files
files.download("race_ru_train.txt")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

## Train 


In [ ]:
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python ru-gpts/pretrain_transformers.py \
    --output_dir=models/RuRACE \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=race_ru_train.txt \
    --do_eval \
    --eval_data_file=race_ru_valid.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 5 \
    --block_size 2048 \
    --overwrite_output_dir

Выходные данные были обрезаны до нескольких последних строк (5000).
Iteration:  25% 443/1802 [07:08<21:37,  1.05it/s]
Iteration:  25% 444/1802 [07:09<21:36,  1.05it/s]
Iteration:  25% 445/1802 [07:10<21:34,  1.05it/s]
Iteration:  25% 446/1802 [07:11<21:33,  1.05it/s]
Iteration:  25% 447/1802 [07:12<21:33,  1.05it/s]
Iteration:  25% 448/1802 [07:13<21:31,  1.05it/s]
Iteration:  25% 449/1802 [07:14<21:30,  1.05it/s]
Iteration:  25% 450/1802 [07:15<21:28,  1.05it/s]
Iteration:  25% 451/1802 [07:16<21:27,  1.05it/s]
Iteration:  25% 452/1802 [07:17<21:27,  1.05it/s]
Iteration:  25% 453/1802 [07:18<21:25,  1.05it/s]
Iteration:  25% 454/1802 [07:19<21:25,  1.05it/s]
Iteration:  25% 455/1802 [07:20<21:23,  1.05it/s]
Iteration:  25% 456/1802 [07:21<21:23,  1.05it/s]
Iteration:  25% 457/1802 [07:22<21:22,  1.05it/s]
Iteration:  25% 458/1802 [07:23<21:20,  1.05it/s]
Iteration:  25% 459/1802 [07:24<21:21,  1.05it/s]
Iteration:  26% 460/1802 [07:25<21:20,  1.05it/s]
Iteration:  26% 461/1802 [07:26<

## Evaluate model
### Generate with generate_transformers.py script.

In [ ]:
!python ru-gpts/generate_transformers.py \
    --model_type=gpt2 \
    --model_name_or_path=models/RuRACE \
    --k=5 \
    --p=0.95 \
    --length=500 \
    --repetition_penalty=5

ТЕКСТ: Загулял наш конюх. Поехал в райцентр вставлять зубы и по случаю завершения такого важнейшего дела загулял. Рейсовый автобус ушёл, и он остался ночевать у свояка.  Кони (их было семеро - два мерина, две кобылы и трое жеребят) долго бродили по лугу, и когда я шёл от реки с удочками, вскинули головы и долго смотрели мне вслед, думая, что, может, я вернусь и загоню их в стойла конюшни, но не дождавшись никого, сами явились в деревню, ходили от дома к дому, и я решил, что они уснут на лугах или прижавшись к стене конюшни,нагретой солнцем со дня.  Поздней ночью я проснулся, пошёл на кухню попить квасу. Что-то остановило меня, заставило глянуть в окно.  Густой-прегустой туман окутал деревню, далее которой вовсе ничего не было видно, и в этой туманной пелене темнели недвижные, как бы из камня вытесанные, силуэты лошадей. Мерины и кобылы стояли, обнявшись шеями, а в серёдке, меж их теплых боков, опустив головёнки, хвосты и жёлтенькие, ещё коротенькие гривы, стояли и спали тонконогие жере

### Generate mannualy

In [ ]:
import numpy as np
import torch

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
tok = GPT2Tokenizer.from_pretrained("models/RuRACE")

In [ ]:
model = GPT2LMHeadModel.from_pretrained("models/RuRACE")

In [ ]:
model.cuda()

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(2048, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
text = "ТЕКСТ: Загулял наш конюх. Поехал в райцентр вставлять зубы и по случаю завершения такого важнейшего дела загулял. Рейсовый автобус ушёл, и он остался ночевать у свояка.  Кони (их было семеро - два мерина, две кобылы и трое жеребят) долго бродили по лугу, и когда я шёл от реки с удочками, вскинули головы и долго смотрели мне вслед, думая, что, может, я вернусь и загоню их в стойла конюшни, но не дождавшись никого, сами явились в деревню, ходили от дома к дому, и я решил, что они уснут на лугах или прижавшись к стене конюшни,нагретой солнцем со дня.  Поздней ночью я проснулся, пошёл на кухню попить квасу. Что-то остановило меня, заставило глянуть в окно.  Густой-прегустой туман окутал деревню, далее которой вовсе ничего не было видно, и в этой туманной пелене темнели недвижные, как бы из камня вытесанные, силуэты лошадей. Мерины и кобылы стояли, обнявшись шеями, а в серёдке, меж их теплых боков, опустив головёнки, хвосты и жёлтенькие, ещё коротенькие гривы, стояли и спали тонконогие жеребята.  Я тихо приоткрыл окно, в створку хлынула прохлада, за поскотиной, совсем близко, бегал и крякал коростель; в ложку и за рекой Кубеной пели соловьи, и какой-то незнакомый звук, какое-то хрюканье, утробное и мерное, доносилось ещё. Не сразу, но я догадался, что это хрипит у самого старого, надсаженного мерина в сонно распустившемся нутре.  Время от времени храп прекращался, мерин приоткрывал чуть смеженные глаза, переступал с ноги на ногу, настороженно вслушиваясь - не разбудил ли кого, не потревожил ли, - ещё плотнее вдавливал свой бугристо вздутый живот в табунок и, сгрудив жеребяток, успокаивался, по-человечьи протяжно вздыхал и снова погружался в сон.  Другие лошади, сколь я ни смотрел на них, ни разу не потревожились, не пробудились и только плотнее и плотнее жались друг к дружке, обнимались шеями, грели жеребят, зная, что раз в табуне есть старшой, он и возьмет на себя главную заботу - сторожить их, спать вполусон, следить за порядком. Коли потребуется, он и разбудит вcex, поведет куда надо. А ведь давно не мужик и не муж этим кобылам старый заезженный мерин, давно его облегчили люди и как будто избавили от надобностей природы, обрекли на уединенную, бирючью жизнь. Но вот поди ж ты, нет жеребцов в табуне - и старый мерин, блюдя какой-то там неведомый закон или зов природы, взял на себя семейные и отцовские заботы. ВОПРОСЫ: "
inpt = tok.encode(text, return_tensors="pt")

In [ ]:
out = model.generate(inpt.cuda(), max_length=800, repetition_penalty=5.0, do_sample=True, top_k=5, top_p=0.95, temperature=1.2)

NameError: ignored

In [ ]:
import os
os.listdir()

['.config', 'sample_data']

In [ ]:
tok.decode(out[0])

'ТЕКСТ: Загулял наш конюх. Поехал в райцентр вставлять зубы и по случаю завершения такого важнейшего дела загулял. Рейсовый автобус ушёл, и он остался ночевать у свояка.  Кони (их было семеро - два мерина, две кобылы и трое жеребят) долго бродили по лугу, и когда я шёл от реки с удочками, вскинули головы и долго смотрели мне вслед, думая, что, может, я вернусь и загоню их в стойла конюшни, но не дождавшись никого, сами явились в деревню, ходили от дома к дому, и я решил, что они уснут на лугах или прижавшись к стене конюшни,нагретой солнцем со дня.  Поздней ночью я проснулся, пошёл на кухню попить квасу. Что-то остановило меня, заставило глянуть в окно.  Густой-прегустой туман окутал деревню, далее которой вовсе ничего не было видно, и в этой туманной пелене темнели недвижные, как бы из камня вытесанные, силуэты лошадей. Мерины и кобылы стояли, обнявшись шеями, а в серёдке, меж их теплых боков, опустив головёнки, хвосты и жёлтенькие, ещё коротенькие гривы, стояли и спали тонконогие жер